In [46]:
import hyperopt
print(hyperopt.__version__)

import xgboost
print(xgboost.__version__)

import lightgbm
print(lightgbm.__version__)

0.2.7
1.5.0
3.3.2


# HyperOpt를 이용한 하이퍼파라미터 튜닝

1. 검색 공간 설정
2. 대체 모델을 위한 목적 함수 지정
3. 최적의 파라미터를 유추

## 1. 검색 공간


In [47]:
from hyperopt import hp

# -10 ~ 10까지 1간격을 가지는 입력 변수 x와 -15 ~ 15까지 1간격으로 입력 변수 y 설정
search_space = {'x': hp.quniform('x',-10,10,1), 
                'y': hp.quniform('y',-15,15,1)}

## 2. 목적 함수 지정

In [48]:
from hyperopt import STATUS_OK

# 목적 함수를 생성. 변숫값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    return retval



## 3. 최적 입력값을 유추

In [49]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# Step 3: 최적 입력값 유추
# 입력 결괏값을 저장한 Trials 객체값 생성.
trial_val = Trials()

# Step 4: TPE 알고리즘을 사용하여 최적화를 수행
# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(
    fn=objective_func,               # 목적 함수
    space=search_space,              # 검색 공간
    algo=tpe.suggest,                # 탐색 알고리즘 (TPE)
    max_evals=5,                     # 최대 시도 횟수
    trials=trial_val,                # Trials 객체
    rstate=np.random.default_rng(seed=0)  # 랜덤 시드
)

print('best:', best_01)

100%|██████████| 5/5 [00:00<00:00, 416.67trial/s, best loss: -224.0]
best: {'x': -4.0, 'y': 12.0}


In [50]:
# max_evals를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest,
               max_evals=20, trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best:', best_02)

100%|██████████| 20/20 [00:00<00:00, 514.88trial/s, best loss: -296.0]
best: {'x': 2.0, 'y': 15.0}


In [51]:
# fmin( )에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨
# 리스트 내부의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값} 와 같은 딕셔너리임. 
trial_val.results

[{'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -64.0, 'status': 'ok'},
 {'loss': -184.0, 'status': 'ok'},
 {'loss': 56.0, 'status': 'ok'},
 {'loss': -224.0, 'status': 'ok'},
 {'loss': 61.0, 'status': 'ok'},
 {'loss': -296.0, 'status': 'ok'},
 {'loss': -40.0, 'status': 'ok'},
 {'loss': 281.0, 'status': 'ok'},
 {'loss': 64.0, 'status': 'ok'},
 {'loss': 100.0, 'status': 'ok'},
 {'loss': 60.0, 'status': 'ok'},
 {'loss': -39.0, 'status': 'ok'},
 {'loss': 1.0, 'status': 'ok'},
 {'loss': -164.0, 'status': 'ok'},
 {'loss': 21.0, 'status': 'ok'}]

In [52]:

# Trials 객체의 vals 속성에 {'입력변수명':개별 수행 시마다 입력된 값 리스트} 형태로 저장됨.
print(trial_val.vals)


{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, -6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0]}


In [53]:
trial_val.best_trial

{'state': 2,
 'tid': 10,
 'spec': None,
 'result': {'loss': -296.0, 'status': 'ok'},
 'misc': {'tid': 10,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'x': [10], 'y': [10]},
  'vals': {'x': [2.0], 'y': [15.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2025, 1, 3, 11, 31, 55, 192000),
 'refresh_time': datetime.datetime(2025, 1, 3, 11, 31, 55, 192000)}

In [54]:
import pandas as pd

# results에서 loss 키값에 해당하는 밸류들을 추출하여 list로 생성
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성
result_df = pd.DataFrame({'x': trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses': losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,-6.0,5.0,-64.0
6,-4.0,10.0,-184.0
7,4.0,-2.0,56.0
8,-4.0,12.0,-224.0
9,9.0,1.0,61.0


# XGBoost 하이퍼파라미터 최적화

In [55]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, Trials, hp
from hyperopt import hp

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

In [56]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )

In [57]:
# 1. 검색 공간
from hyperopt.pyll.base import scope

# 하이퍼 파라미터 검색 공간
search_space = {'n_estimators':scope.int(hp.quniform('n_estimators', 50,300,10)),
                'max_depth': scope.int(hp.quniform('max_depth', 3, 10, 1)), 
                'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.3)),
                'subsample': hp.uniform('subsample', 0.5,1.0),
                'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
                }


# 교재
# from hyperopt import hp

# # max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# # colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색.
# xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1), 
#                     'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
#                     'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
#                     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
#                    }

In [58]:
# 2. 목적 함수
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

def objective_func_xgb(params):
    model = XGBClassifier(
        n_estimators=params['n_estimators'], 
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=42,
        eval_metric='logloss', 
    )
    score_mean = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss': -1*score_mean , 'status': STATUS_OK}


# 교재
# from sklearn.model_selection import cross_val_score
# from xgboost import XGBClassifier
# from hyperopt import STATUS_OK

# # fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형임.
# # XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# # 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환
# def objective_func(search_space):
#     # 수행 시간 절약을 위해 nestimators는 100으로 축소
#     xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
#                             min_child_weight=int(search_space['min_child_weight']),
#                             learning_rate=search_space['learning_rate'],
#                             colsample_bytree=search_space['colsample_bytree'],
#                             eval_metric='logloss')
#     accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    
#     # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
#     return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [59]:
# 3. 최적 입력값을 유추
from hyperopt import fmin, tpe, Trials
import warnings
warnings.filterwarnings('ignore')

# Hyperopt Trials 초기화
trial_val = Trials()
# Hyperparameter Tuning 실행
best_params = fmin(fn=objective_func_xgb,
            space=search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val,
            #rstate=np.random.default_rng(seed=9)
            )

print('best:', best_params)

# 교재
# from hyperopt import fmin, tpe, Trials
# import warnings
# warnings.filterwarnings('ignore')

# trial_val = Trials()
# best = fmin(fn=objective_func,
#             space=xgb_search_space,
#             algo=tpe.suggest,
#             max_evals=50, # 최대 반복 횟수를 지정합니다.
#             trials=trial_val, 
#             rstate=np.random.default_rng(seed=9))
# print('best:', best)

100%|██████████| 50/50 [01:07<00:00,  1.34s/trial, best loss: -0.9714285714285715]
best: {'colsample_bytree': 0.7640743223872615, 'learning_rate': 0.16871176836822976, 'max_depth': 5.0, 'n_estimators': 120.0, 'subsample': 0.5041991491400559}


In [60]:
best

{'colsample_bytree': 0.5424149213362504,
 'learning_rate': 0.12601372924444681,
 'max_depth': 17.0,
 'min_child_weight': 2.0}

## 최적의 파라미터를 적용한 모델 생성

In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_probs=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred) 
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred)

    print('오차 행렬:\n', confusion)
    print(f'정확도: {accuracy}, 정밀도: {precision}, 재현율: {recall},\
    F1: {f1}, AUC:{roc_auc}')

In [62]:
best_model = XGBClassifier(n_estimators= int(best_params['n_estimators']),
                           max_depth= int(best_params['max_depth']),
                           learning_rate= best_params['learning_rate'],
                           subsample= best_params['subsample'],
                           colsample_bytree= best_params['colsample_bytree'],
                           random_state= 42,
                           eval_metric= 'logloss'
                           )

from sklearn.metrics import accuracy_score

best_model.fit(X_train, y_train)

pred= best_model.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('XGBoost+HyperOpt accuracy', accuracy)

#교재
# xgb_wrapper = XGBClassifier(n_estimators=400,
#                             learning_rate=round(best['learning_rate'], 5),
#                             max_depth=int(best['max_depth']),
#                             min_child_weight=int(best['min_child_weight']),
#                             colsample_bytree=round(best['colsample_bytree'], 5)
#                            )

# evals = [(X_tr, y_tr), (X_val, y_val)]
# xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss',
#                 eval_set=evals, verbose=True)

# preds = xgb_wrapper.predict(X_test)
# pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]


# get_clf_eval(y_test, preds, pred_proba)
# print('###XGBoost+HyperOpt###')    

XGBoost+HyperOpt accuracy 0.9736842105263158
